# Assignment 4b: Linguistic statistics

### This assignment is due 5-12-2017, before 20:00 PM. 
   
The goal of this assignment is to compute statistics on the basis of a corpus, in order to compare it with other corpora. This is useful, because current natural language processing technology is quite sensitive to these differences. A NLP system that is trained on one kind of corpus, and displays very good performance, might perfom much worse on a different kind of corpus. This is known as the problem of *domain adaptation*. 

## Goal
The goal of this exercise is to complete the function below this cell.

Aside from being able to compute the statistics, you will also be graded on your use of helper functions. For example, it would be reasonable not to use two separate helper functions in order to computer **num_tokens** and **num_sents** since they both require you to loop over the **wf elements**.

In [1]:
path_to_naf_file = '../Data/xml_data/naf.xml'

In [ ]:
def extract_stats(path_to_naf_file):
    """
    given the path to a NAF XML file
    this function will complete the dictionary with linguistic stats
    
    :rtype: dict
    :return: dictionary with linguistic stats
    """
    # your code here (calling of helper functions)
    
    
    stats = {
        'num_tokens'    :          , # number of wf elements in naf file (see wf elements)
        'num_sents'     :          , # number of sentences in naf file (see wf elements)
        'longest_word'  :          , # the longest word (see wf elements)
        'longest_sent'  :          , # the longest sentence (see wf elements)
        'pos_freq'      :          , # frequency of each pos (see term elements attribute 'pos')
        
        'num_entities'  :          , # number of entities (see entity elements)
        'type_freq'     :          , # frequency of each entity type (see entity element attribute type)
        
    }